In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn import svm
from sklearn.datasets import make_blobs
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Load Excel file
df = pd.read_csv("mlb_game_data_2025.csv")

# Explore the structure
print(df.shape)
print(df.head(5))

(2463, 21)
             Date Home Team Away Team  Difference doubles  Difference triples  \
0  March 27, 2025       NYY       MIL                 0.0                 0.0   
1  March 27, 2025       TOR       BAL                -1.0                 0.0   
2  March 27, 2025       TEX       BOS                 1.0                 0.0   
3  March 27, 2025       WSH       PHI                -1.0                -1.0   
4  March 27, 2025        KC       CLE                -1.0                -1.0   

   Difference homeruns  Difference hits  Difference hitbypitch  \
0                  1.0              0.0                    0.0   
1                 -5.0            -10.0                   -1.0   
2                 -2.0              1.0                    0.0   
3                 -1.0             -3.0                    1.0   
4                  0.0             -3.0                    0.0   

   Difference avg  Difference obp  ...  Difference ops  \
0           0.029           0.035  ...         